In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import pandas as pd
import geopandas as gpd

In [ ]:
import unicodedata


def text_normalize(text: str) -> str:
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8")
    text = text.lower()
    return text


text = "Local no construido para habitación"
print(f"`{text}` -> `{text_normalize(text)}`")

# Importar data

In [ ]:
CENSAR_DATA_URI = os.getenv(
    "CENSAR_DATA_URI", "https://storage.googleapis.com/python_mdg/censar_data"
)
CENSAR_CARTO_URI = os.getenv(
    "CENSAR_CARTO_URI", "https://storage.googleapis.com/python_mdg/censar_carto"
)


def tipoviv_radios_prov(year, prov, var_types, root=CENSAR_DATA_URI):
    path = f"{root}/tipo_vivienda_radios_{prov}_{year}.csv"
    print(path)
    tipoviv_radio = pd.read_csv(path, dtype=var_types)
    tipoviv_radio.columns = [text_normalize(col) for col in tipoviv_radio.columns]
    return tipoviv_radio


viviendas_2001 = tipoviv_radios_prov(
    year=2001,
    prov="corrientes",
    var_types={"LINK": "object"},
    root=CENSAR_DATA_URI,
)
viviendas_2010 = tipoviv_radios_prov(
    year=2010,
    prov="corrientes",
    var_types={"link": "object"},
    root=CENSAR_DATA_URI,
)
viviendas_2010

# Importar radios

In [ ]:
# FIXME: Reemplazar por funcion de CENSAr.datasources
def radios_prov(year, prov, root=CENSAR_CARTO_URI, mask=None):
    path = f"{root}/radios_{year}_{prov}.zip"
    radios = gpd.read_file(path)

    if mask is not None:
        if mask.crs != radios.crs:
            mask = mask.to_crs(radios.crs)
        radios = radios.clip(mask)

    radios.columns = [text_normalize(col) for col in radios.columns]
    return radios


# MASCARA (Esta mascara es provisioria. Lo correcto sería poder utilizar el límite vectorial del extent de la mancha urbana detectada en una imagen landsat-sentinel)
path = "/resources/data/radios_eph_json.zip"
mask = gpd.read_file(path)

mask_wgs = mask[mask["eph_codagl"].isin(["12"])].copy().to_crs(4326)
mask_wgs["cons"] = 0
envolvente = mask_wgs.dissolve(by="cons")

radios_2001 = radios_prov(
    year=2001,
    prov="corrientes",
    mask=envolvente,
    root=CENSAR_CARTO_URI,
)
radios_2010 = radios_prov(
    year=2010,
    prov="corrientes",
    mask=envolvente,
    root=CENSAR_CARTO_URI,
)
radios_2010

# merge indicadores + radios

In [ ]:
data_2001 = radios_2001.merge(viviendas_2001, on="link")
data_2010 = radios_2010.merge(viviendas_2010, on="link")
data_2001.plot(column="rancho", legend=True)
data_2010.plot(column="rancho", legend=True)
data_2010

# Agregacion de campos

In [ ]:
from CENSAr.aggregation.core import aggregate

schema = [
    {
        "name": "Informal",
        "columns": ["rancho", "casilla", "local no construido para habitacion"],
    },
    {
        "name": "Formal",
        "columns": [
            "departamento",
            "pieza en inquilinato",
            "pieza en hotel o pension",
            "vivienda movil",
        ],
        "regex": ["casa *"],
    },
    {
        "name": "Situacion de calle",
        "columns": ["en la calle"],
    },
]

tipo_vivienda_particular_2001 = aggregate(data_2001, schema)
tipo_vivienda_particular_2010 = aggregate(data_2010, schema)

In [ ]:
from CENSAr.aggregation import list_named_aggregations

list_named_aggregations()

In [ ]:
from CENSAr.aggregation import named_aggregation

vulnerabilidad_2001 = named_aggregation(
    tipo_vivienda_particular_2001, "Vulnerabilidad Urbana"
)
vulnerabilidad_2010 = named_aggregation(
    tipo_vivienda_particular_2010, "Vulnerabilidad Urbana"
)

# Plots

In [ ]:
from CENSAr.plots import choropleth, compare_chropleths, compare_columns_chropleths

fig = choropleth(vulnerabilidad_2001, "No vulnerable")
fig.show()

In [ ]:
compare_columns_chropleths(vulnerabilidad_2001, ["Vulnerable", "No vulnerable"]).show()

In [ ]:
compare_chropleths(vulnerabilidad_2001, vulnerabilidad_2010, column="Vulnerable").show()